In [1]:
gameFolder = "UMK3-MAME"

In [2]:
import sys, os
import time
timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../'))   
sys.path.append(os.path.join(os.path.abspath(''), '../../../games',gameFolder))   

tensorBoardFolder = "./ppo2_TB_CustCnn/"
modelFolder = "ppo2_Model_CustCnn/"

os.makedirs(modelFolder, exist_ok=True)

In [3]:
from makeDiambraEnv import *

import tensorflow as tf

from customPolicies.utils import linear_schedule
from customPolicies.customCnnPolicy import *

from stable_baselines import PPO2

In [4]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../../customMAME/"
diambraKwargs["player"] = "P1"
diambraKwargs["frame_ratio"] = 3
diambraKwargs["render"] = False
#diambraKwargs["throttle"] = False
#diambraKwargs["sound"] = False 
#diambraKwargs["character"] ="Random"
diambraKwargs["character"] = "Sektor"

wrapperKwargs = {}
wrapperKwargs["hwc_obs_resize"] = [256, 256, 1]
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["frame_stack"] = 6
wrapperKwargs["scale"] = True
wrapperKwargs["scale_mod"] = 0


#keyToAdd = None
keyToAdd = []
keyToAdd.append("actionsBuf")
#keyToAdd.append("player")
keyToAdd.append("healthP1")
keyToAdd.append("healthP2")
keyToAdd.append("positionP1")
keyToAdd.append("positionP2")
#keyToAdd.append("winsP1")
#keyToAdd.append("winsP2")

numEnv=4

env = make_diambra_env(diambraMame, env_prefix="Train", num_env=numEnv, seed=timeDepSeed, 
                       continue_game=0, diambra_kwargs=diambraKwargs, 
                       wrapper_kwargs=wrapperKwargs, key_to_add=keyToAdd)


Env_id =  Train0
Continue value =  0
Player = P1 , Character = Sektor
Noop action N =  14
Env_id =  Train1
Continue value =  0


/home/apalmas/anaconda3/envs/diambra/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Player = P1 , Character = Sektor
Noop action N =  14
Env_id =  Train2
Continue value =  0
Player = P1 , Character = Sektor
Noop action N =  14
Env_id =  Train3
Continue value =  0
Player = P1 , Character = Sektor
Noop action N =  14


In [5]:
print("Obs_space = ", env.observation_space)
print("Obs_space type = ", env.observation_space.dtype)
print("Obs_space high = ", env.observation_space.high)
print("Obs_space low = ", env.observation_space.low)

Obs_space =  Box(256, 256, 7)
Obs_space type =  float32
Obs_space high =  [[[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 ...

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 

In [6]:
print("Act_space = ", env.action_space)
print("Act_space type = ", env.action_space.dtype)
print("Act_space n = ", env.action_space.n)

Act_space =  Discrete(15)
Act_space type =  int64
Act_space n =  15


In [7]:
# Policy param
policyKwargs={}
policyKwargs["n_add_info"] = 184
policyKwargs["layers"] = [64, 64]

# Initialize the model, 1 env
model = PPO2(CustCnnPolicy, env, verbose=1, 
             gamma = 0.94, nminibatches=4, noptepochs=4, n_steps=128,
             learning_rate=linear_schedule(2.5e-4, 2.5e-5), cliprange=linear_schedule(0.2, 0.1),
             tensorboard_log=tensorBoardFolder, policy_kwargs=policyKwargs)

#OR

# Load the trained agent, 1 env
#model = PPO2.load(modelFolder+"4_5Msteps_action+_add", env=env, tensorboard_log=tensorBoardFolder, 
#                  policy_kwargs=policyKwargs, gamma = 0.94)





Instructions for updating:
Use keras.layers.flatten instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [ ]:
# Train the agent
time_steps = 20000000
model.learn(total_timesteps=time_steps)


Setting difficulty = 4
Starting game
Selecting Sektor
Waiting for fight to start
Setting difficulty = 4
Starting game
Selecting Sektor
Waiting for fight to start
Setting difficulty = 4
Starting game
Selecting Sektor
Waiting for fight to start
Setting difficulty = 4
Starting game
Selecting Sektor
Waiting for fight to start
--------------------------------------
| approxkl           | 0.00020415732 |
| clipfrac           | 0.0           |
| explained_variance | -2.46         |
| fps                | 9             |
| n_updates          | 1             |
| policy_entropy     | 2.7078543     |
| policy_loss        | -0.0034307693 |
| serial_timesteps   | 128           |
| time_elapsed       | 2.17e-05      |
| total_timesteps    | 512           |
| value_loss         | 1.2586591     |
--------------------------------------
--------------------------------------
| approxkl           | 0.0003751993  |
| clipfrac           | 0.0           |
| explained_variance | -1.96         |
| fps       

Episode done
Restarting game
Selecting Sektor
Waiting for fight to start
Episode done
Restarting game
Selecting Sektor
Waiting for fight to start
-------------------------------------
| approxkl           | 0.009885879  |
| clipfrac           | 0.14208984   |
| ep_len_mean        | 905          |
| ep_reward_mean     | -3           |
| explained_variance | -0.0927      |
| fps                | 13           |
| n_updates          | 15           |
| policy_entropy     | 2.5619233    |
| policy_loss        | -0.034709673 |
| serial_timesteps   | 1920         |
| time_elapsed       | 422          |
| total_timesteps    | 7680         |
| value_loss         | 0.0038883367 |
-------------------------------------
Episode done
Restarting game
Selecting Sektor
Waiting for fight to start
-------------------------------------
| approxkl           | 0.011432188  |
| clipfrac           | 0.17089844   |
| ep_len_mean        | 950          |
| ep_reward_mean     | -3.03        |
| explained_variance 

In [ ]:
# Save the agent
model.save(modelFolder+"5_8Msteps_action+_add")

In [ ]:
observation = env.reset()
states = None

while True:

    action, states = model.predict(observation, states, deterministic=False)
    action_prob = model.action_probability(observation, states)
    print("Action probabilities = ", action_prob)
    print("Max action = ", np.argmax(action_prob))
    print("Action = ", action)
    input("Pausa")
    
    observation, reward, done, info = env.step(action)
    if done:
        observation = env.reset()
        states = None
        
env.close()

In [ ]:
observation = env.reset()
states = None

cumulativeEpRew = 0.0
cumulativeEpRewAll = []
cumulativeTotRew = 0.0

maxNumEp = 100
currNumEp = 0

while currNumEp < maxNumEp:

    action, states = model.predict(observation, states, deterministic=False)
    action_prob = model.action_probability(observation, states)
    print("Action probabilities = ", action_prob)
    print("Max action = ", np.argmax(action_prob))
    print("Action = ", action)
    
    observation, reward, done, info = env.step(action)
    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        sys.stdout.flush()
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeTotRew += cumulativeEpRew
        cumulativeEpRew = 0.0

        observation = env.reset()
        states = None

print("Mean cumulative reward = ", cumulativeTotRew/maxNumEp)    
print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))   
    
env.close()